# PARTIE A : IMPLEMENTATION DU JEU

In [171]:
import numpy as np
import pygame
import heapq

dim = 0
k = 0
while dim == 0 or dim > 5 or k == 0 :
    print("Enter la dimension (inférieur à 5): ", end="")  # end="" empêche le saut de ligne automatique
    # Lire l'entrée utilisateur (équivalent de scanf)
    dim = int(input())
    print("Choisir votre nombre de deplacement : ", end="")
    k = int(input())

# Creer un matrice aleatoire
numbers = np.random.choice(np.arange(1,101), size=dim**2, replace=False)

# Reshaper les nombres mélangés en une matrice 3x3
matrix = numbers.reshape(dim, dim)

# Choisir une position aléatoire pour placer un 0
row, col = np.random.randint(0, dim), np.random.randint(0, dim)
matrix[row, col] = 0

# Recherche la position vide
p = np.where(matrix == 0)
lig_vide = p[0][0]
col_vide = p[1][0]

#print(matrix)
#print("ligne : ",lig_vide)
#print("colonne : ",col_vide)
#print("k :", k)

Enter la dimension (inférieur à 5): 

 6


Choisir votre nombre de deplacement : 

 2


Enter la dimension (inférieur à 5): 

 5


Choisir votre nombre de deplacement : 

 3


# PARTIE B: Recherche des successeurs de l'element vide à base de l'Algorithme A*

In [173]:
def get_successors(noeud, matrix_size):    
    x, y = noeud
    successors = []
    
    # Déplacements possibles : 8 directions (haut, bas, gauche, droite, diagonales)
    mouvements = [
        (-1, 0), (1, 0),  # Haut, Bas
        (0, -1), (0, 1),  # Gauche, Droite        
    ]
    
    for dx, dy in mouvements:
        nx, ny = x + dx, y + dy
        
        # Vérifier que la nouvelle position est dans les limites de la matrice
        if 0 <= nx < matrix_size and 0 <= ny < matrix_size:
            successors.append((nx, ny))
    
    return successors

# Exemple d'utilisation
dim_matrix = dim  # Taille de la matrice 5x5
noeud_courant = (lig_vide, col_vide)  # Position actuelle
successeurs = get_successors(noeud_courant, dim_matrix)

print(f"Successeurs de {noeud_courant} : {successeurs}")

Successeurs de (1, 4) : [(0, 4), (2, 4), (1, 3)]


# EVENEMENT D'ECOUTE DE L'UTILISATEUR
# Partie jouable par l'utilisateur

In [175]:
# pygame setup
pygame.init()

screen = pygame.display.set_mode((1020, 620))
clock = pygame.time.Clock()
running = True

LARGEUR, HAUTEUR = 1020, 620

CELLULE_LARGEUR = LARGEUR // matrix.shape[1]
CELLULE_HAUTEUR = HAUTEUR // matrix.shape[0]

NOIR = (0, 0, 0)
BLANC = (255, 255, 255)
ROUGE = (0, 255, 0)  # Couleur rouge pour la cellule sélectionnée


# Changer la couleur de la cellule sélectionnée
cellule_colorier = (lig_vide, col_vide)
nouvelle_couleur = ROUGE

police = pygame.font.Font(None, 36)

while running:
    # poll for events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False    
        # Vérifier si une touche est enfoncée
        elif event.type == pygame.KEYDOWN:
            if (event.key == pygame.K_UP) and lig_vide > 0:    
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide-1, col_vide
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]                
                lig_vide -= 1                   
            if (event.key == pygame.K_DOWN) and lig_vide < dim - 1:   
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide+1, col_vide
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]
                lig_vide += 1                   
            if (event.key == pygame.K_LEFT) and col_vide > 0:   
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide, col_vide-1
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]
                col_vide -= 1                   
            if (event.key == pygame.K_RIGHT) and col_vide < dim - 1:       
                i1, j1 = lig_vide, col_vide 
                i2, j2 = lig_vide, col_vide+1
                matrix[i1, j1], matrix[i2, j2] = matrix[i2, j2], matrix[i1, j1]
                col_vide += 1                   

            # Mettre à jour la cellule colorée
            cellule_colorier = (lig_vide, col_vide)

    # Remplissage de l'écran
    screen.fill(NOIR)

    # Dessin du tableau
    for i in range(matrix.shape[0]):  # Parcourt les lignes
        for j in range(matrix.shape[1]):  # Parcourt les colonnes
            # Coordonnées du coin supérieur gauche de la cellule
            x = j * CELLULE_LARGEUR
            y = i * CELLULE_HAUTEUR

            # Changer la couleur de la cellule sélectionnée
            if (i, j) == cellule_colorier:
                color = nouvelle_couleur
                text_color = NOIR  # Texte en noir pour la cellule rouge
            else:
                color = BLANC
                text_color = NOIR  # Texte en noir pour toutes les autres cellules

            # Rectangle pour chaque cellule (fond de la cellule)
            pygame.draw.rect(screen, color, (x, y, CELLULE_LARGEUR, CELLULE_HAUTEUR))

            # Rectangle pour le contour des cellules (en blanc)
            pygame.draw.rect(screen, BLANC, (x, y, CELLULE_LARGEUR, CELLULE_HAUTEUR), 1)

            # Texte à afficher dans la cellule
            texte = police.render(str(matrix[i, j]), True, text_color)
            texte_rect = texte.get_rect(center=(x + CELLULE_LARGEUR // 2, y + CELLULE_HAUTEUR // 2))
            screen.blit(texte, texte_rect)

    # Mettre à jour l'affichage
    pygame.display.flip()

    # Limiter les FPS à 60
    clock.tick(60)

pygame.quit()